In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

In [2]:
# Dato I/O

In [3]:
data = open('shakespeare.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 99992 characters, 62 unique.


# Model and Hyperparameters

In [4]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias




In [9]:
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [ ]:


n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    
    
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print ('----\n %s \n----' % (txt, ))
  # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress

    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 qCERoWG ?A;wKgdNakrMJhHk!JPUIvB?xYWoEjoJKqTdgJecHIH.vFTlQDFclsKI-XfrGqZp-.eKu
AfEgTWHD,sjtkyBGRaTRYVjH?z'GhpFHojMSgWak mVuYsk:glLMfYspQOCfvausl'p'-vlwgiIvrPPNnGVwXKbE'qndsvLuJ
WT?:z;,noc!tkxnIMFKpjIIp 
----
iter 0, loss: 103.178357
----
 td
 r:iokm gduIg A trn  ushetybat
.k r
 eont y re eeawdsih n a,t ic ufa
d fe hc Adrit A
srpousb ReeSgtnh
r ta nrrTpaad A
r
m  dS st eaa  idow
su ae ItpfAy 
l tt ee 
s  SfR  dt olo:no,
sIe hbmGbau
 :yd 
----
iter 100, loss: 103.154869
----
  rh eosmiTDb

 eyoIi  FdfenayudlSty estsroTofhloeandoomldeae koonimad esOaeogweenIlhoro et ae Do e  aetaudhshii
pe mieaooit ao uoIa
Np taeraeeen afma ahlueusIeta G
sdh'essvso eeoarM delnx ooho: o hhds 
----
iter 200, loss: 101.849087
----
 o usaa Ie,itc,Ay ye e D.
lhd 
mIad ERMArret sdnarehit.tei: lesmo h  toCotfj Sseoeg nbSsIuR, oIt t l Doeurrhsi yn el n:es  ir hrsptOpI

 tp neTfs .eLHoTh 
uot y  hcofjuwr.se.e lWetacsfe pwTw hEh,to ss  
----
iter 300, loss: 100.371064
----
 mb
sdrea
ERc
Y Ifus,t    AhetIdna dv

----
 ro lldith mom; wus
I Allas
d verd on
Bisct on trin'd ah, thdid bvaageleu fou le shes int t aun yevens;
Ooc: ci ss thi westous
Bu peu il,
Bcy ole hat fey ar bosuf, se lirse It Zid wor to-d saranes kiti 
----
iter 3500, loss: 66.007269
----
 ou shive weiut, thy prer, lakhsildend thot hee an go I I:
S pinlse thspisgs oh avy ood whpas tn ean;s tonfe on!, wott wabt mor sord yhax, rut buld um douik.
Aren I adt Mas che ve; ice.
wit ther semege 
----
iter 3600, loss: 65.595371
----
 cas anpt shawins ojocr ineceea moun ons uil, bnond orime.

PyF:
Rfon; houf Thawlad out gfun? I irt ind ofamel chert'qd sey

As:
AKBU
AKS:;
Yit het hasce yordafhoi see, sassyin it tea the so cith ko tr 
----
iter 3700, loss: 65.383592
----
 re hags tiy golk t tee pis, yo'd;,
 EBXfaserelnged heurthaw az, hat thive ove cok cerme Yos ibf yuktrus onelsthheis ous

I,
AnI miry ppey hoar'sd ans nou'? I
And,  Cold mwiiml

UPXAUI:
An hyod bashit  
----
iter 3800, loss: 65.060545
----
 wond dinder ir goul ooundton ic,
A

----
 BPUEOOET:
IK I
Her

UVNTILEN:
Torrovel'geell bnite, uthit ondedtart islasd ticbingsdro brelcars theore voal rowat fs
Ande of padsy o lytreth an y.
Tur urcher Sound trel aopty dortnondortertemdes,
TA-e 
----
iter 7000, loss: 59.459085
----
 pow, ar yey: sour,
Ye ind me,
andid? forth my:oxhas sdacteeme steltee sond kime hyt gheede lome no't moray keow, shind als brou nowa
Y:

BOS:
Thet tave at of some nofs did?
Po che al hledd thell the t 
----
iter 7100, loss: 59.374861
----
 e hoFl four

FREI:
The 'ds of worer, loy ar pucll co fund shear.

BHRDETw:
He my fo exHoWas,
My anke, wamh my dttior hy urp, dera, thimit cheuk

LLSEBLILEDR:
Shel janou des indet as woa me mames gladd 
----
iter 7200, loss: 59.354614
----
 tneby woull.

HACAsTJCEY:
An' hoimsal thatces sin! a, be,
Whet Uore gat bral
 koon oS sdan nold?
Pn wled cuvo-s.
Muse
Ant cot, a wand moy af wemand snaite.

ASli:
Menok:
O, hee hent ow thathe.

DSE OE 
----
iter 7300, loss: 59.210828
----
 of.

Band neyus Thath drit saled i

----
 anse

KHIH LMIN,
TIMUN:
Roof the, thorcishqunpingond I nomce theaus 'of thvomerd.

HETIUCINOE:
Thes bet!,
Bwliclaut bovlties;
Mas fine ind.

PRALIY:
Fnyalknoube, momy moe.

Kecharg till pathanm wim al 
----
iter 10500, loss: 57.524245
----
 ur yeat'the, han noo to gorncas auts noull whrotit mer ppsyaoy dand,
I burte cooret fut kFouve yoteiive kpim ptofeteqele salds yor lealgI UR beds, feas spolk thiur yow ut so thas hives:
We
To doo fit  
----
iter 10600, loss: 57.574444
----
 .hy
of, folen
 four stess tisinor;, kor my anin be,
I civestilcouse;
Ald my tar int the.
I Lhaasloue:

B Lcey thaI wo, co bay er, aal of indie il kxthift!

SANETEN: houineller the nou mr ainevets: tho 
----
iter 10700, loss: 57.482110
----
 he:
Wasgee a eh not he, and toul; winghion teesr you thou ansstowe.

Allw tounbowh nou cyell enmt loI Do arll sE culs
powtttall that and peoy sous
The ly mowe lok lasesth ow sreld
Ar akam the bace ind 
----
iter 10800, loss: 57.387636
----
 orhert had ta thald bultirn ir

----
  to thlas ther; in thoss make thicor mis coop thlyelt the vot.

IORA:
Thitt pubt.

CUUGUG PEIUNNHiRG ROBnGsh as them the weryas tore-gor, linssras; to
Thimthon, tesis shediss.

SoLTicd mat moods: urk  
----
iter 13900, loss: 56.071072
----
 k my heace: wicgong.

FALSTMLNRIS AR:
As thy goed the mondsedratstenoll wowndasperter.

SLOD:
Is mrbakith wus gries bhaghhpelend gactter ot for thicly what his the, so ich sengeacusra my Ditlous in bo 
----
iter 14000, loss: 56.126461
----
 f I so fare waon l!

MUBYYANAN:g sheon ip.,
Thay oo sowe hick, as goon, saor: hoan speryall anmes, ow chee an.
And, podfame heayd
Bigh Penur ow yousgeend bes ate ciy geises apkee and croo lis, t? at h 
----
iter 14100, loss: 56.238827
----
 so lareme meant thesgik sotrar, an.

MINS:
Gay to yow Orout me lot, indind is at I thit an bel vour, pomes maler toull Bude care arer.
Rove,
As wade:
SoGs int got enourt,
Int eay arand aun poseredr de 
----
iter 14200, loss: 56.276949
----
 I pxcome wase the mist cer
Bul

...

----
 iet
Botlemstond datp:
Noul will rece.
Thuth roms
 CoPH HE
CISSILI:
Myor I dinet mokegh dall for I eartarn ore my:
That worde,
There prostreie paeffines of the crauld, he cive of
thy's ques entchisy no 
----
iter 574900, loss: 46.367082
----
  you she Cotd day, I I, and whonk thy as, won toim ay Cirsues a wown and: am your on, sor a mad rear hatl Lome,
Inty
Wourd indiit so pensumne lamed peake and mall not sicing hean my to to so feard,
Yo 
----
iter 575000, loss: 46.439647
----
 re heed my coulfor usjortonk crife
And solingert good kirther will
Sean's may to the by alout spart aidles, their lepe pear a
Heve the woursiter,
Whown,
And and prows; wourd pray det of denysred drats 
----
iter 575100, loss: 46.379187
----
 ire: him hee, other the bereran hing;
But lordt ald lean divn try wrave whall say,
 Mang I thaticy and to hay agaif pradian une a truer dack with pracent lord, Saveslurers?
Then murt hy I sto awe-;
sh 
----
iter 575200, loss: 46.307050
----
 e.

Nyo Lom:
If suy ann li

----
 ; most and fore earnt lither.

CLURTN:
Whe her that sas bry, with 've nave hrout contige
And Nifreingt
And lance
ThverI wad rore ass to enses.

KINGlK ChE
ROY:
To dount murk with viing blue?

DARGDUS: 
----
iter 578300, loss: 46.355170
----
 e elrist,
Whome formind,
Which your brielowed derour,
Bise wist me, hast led the ruede of plore my both wornde tit and your on you aple oun up if hond is movespersene the lere fact greirech be
glouds  
----
iter 578400, loss: 46.427057
----
 iffar mack,
But lvay det; hord, theree awe, thy I maught his and Tited nother my thot speed
Yobad loese, hos here's thasesans pright the greazer to bured, ant he.

DUKE OFla

WAlostey then?
Thewvice

 
----
iter 578500, loss: 46.439416
----
 Were and is thus my far kllunet:
of Alloms: but he kings the din be me Slean the raild whose that vomerore seat er'd and it. By do wisy to we deed furd!

ALDUS:
is no suce preaks and. Waceas trese war 
----
iter 578600, loss: 46.446347
----
 t thee.

TETERK:
I he quay

----
 cearyolch
He kive thiss so there nout sweellarts if him frass a Jone nouren hone'.

CRYARK QUESSO:
Cumey
I am hade thy dos, to a sheagh ands:
Thaitir thove intiouk blosher
Aghn it a you coar my lord,
 
----
iter 581700, loss: 45.886551
----
 

BARK OONED:
Nas to to fichen fairret-am my deare:
of wave, ly, esiles, the indy!

BOFFUY:
Ne ank yot her I am thy paspetsly youch?

KENL OLLA:
Tole: 's Tind: his ther have and fored.

KING:

Seresar 
----
iter 581800, loss: 46.131090
----
 ech
Yould
Am Marst you, awnte;
to my fasa hinds at is um am all foon'dice be we intenfore muthed feer enclicerdar, minds you for:
That do not carrean faild to lior a mons.

ROLIUCENNY: Ous thifef is b 
----
iter 581900, loss: 46.170813
----
 thath then, and he thy rordurtef, effent dory if kis nack a Proun.-
Sle, fall was hond hir, I would cont.

Firse this deel thome or now of eet seffe that on scesty Crang I drievel,
Afood!
Ofain the ho 
----
iter 582000, loss: 46.156908
----
 e forenen he it bey of res

----
  a'lif,
Bear, was the gereryolinjiar: my with booy led he moreser, I a meast
A he this
Griepceds: moriss, solothat: with beft biteill all rlone this growe gloce
I wouls; doUs
Sistouch the cunconon.

T 
----
iter 585100, loss: 46.075050
----
 ll,
To mung her.

DOLPEDSSEICH:
Thou rath ports her to thate am of hip deant a vanch inoun my wall!

Lay unt this ang of ithe Yos may youn's temand donted thear.
That fauld ot it a bust of awertail: I 
----
iter 585200, loss: 45.954399
----
 and the the bricks
And the whithy vistro woln of,
Suaks onat dasiin Thisus would shall had feler that all theed, inped'ds dose is.

SICINTON:
He hey so plould vis somh!
Sthem mays, enst det,
Well and  
----
iter 585300, loss: 45.971883
----
 u abty and nearns now and tand.
I a guilty I her
Thight
Thul, of I my pay his up ick Semy your motrere to do froutin:
We grain.

ICIO:
A bespear
Which you you up for I sheren lord allow is which,
To k 
----
iter 585400, loss: 45.924081
----
 y bhats of our maky my din

----
 Well and face on theit, ve of ot snownws preegier know well heat 'no' thean diveres goothly the ppot,
My quearturn me,
That tak,
Agt:
on thiy, eack spot
Swandy is fat, I have yed seange unrmeak
To gre 
----
iter 588500, loss: 46.138526
----
 at whut hea uct a for heade that do Caar?

PENDONM:
Kf and is a-mance of 'Tmonightain of wo who hel's is you have yen kif
ond, bour fefton sweet my swius
shuth then that hcarces prout me to to not yeo 
----
iter 588600, loss: 46.090304
----
 th us told the s'e shour tithor baven buther thee:
bot nyou dorld
Iur loys.

QhAth to ever,
gatimpent I eskescath and of hasises strithe be heald is
Be stol shily,
Plich his Orr is joyefite and thy's  
----
iter 588700, loss: 46.200544
----
 ?

KENT:
And to you.

MARBENEDA:
I ville swisy racuprmerucor paces.f.b Loldise plase thisy
Beer hean'ditly daanty and betiod wrifut tick,
But dits notl wore that lords,
Beante go hear is notshot tith  
----
iter 588800, loss: 46.206796
----
 not merour manaut have bot